In [1]:
import pickle

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error

In [ ]:
# mlflow ui --backend-store-uri sqlite:///mlflow.db

In [3]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2024/11/02 12:51:13 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2', creation_time=1730551873491, experiment_id='2', last_update_time=1730551873491, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
    df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])

    # Calculate duration
    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']

    # in seconds
    df['duration'] = df['duration'].dt.total_seconds() / 60

    # durations for more than 1 minute and less than 60 minutes
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

    categorical = ['PULocationID', 'DOLocationID']

    df[categorical] = df[categorical].astype(str)
    
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    
    return df

In [5]:
train_df = read_dataframe("./data/green_tripdata_2024-01.parquet")
test_df = read_dataframe("./data/green_tripdata_2024-02.parquet")

In [7]:
categorical = ['PU_DO']#['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

train_dict = train_df[categorical + numerical].to_dict(orient="records")
val_dicts = test_df[categorical + numerical].to_dict(orient="records")

dv = DictVectorizer()
X_train = dv.fit_transform(train_dict)
X_val = dv.transform(val_dicts)

target = 'duration'
y_train = train_df[target].values
y_val = test_df[target].values

In [22]:
with mlflow.start_run():
    # mlflow.
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    
    y_train_pred = lr_model.predict(X_train)
    y_pred = lr_model.predict(X_val)
    
    print("train: ", mean_squared_error(y_train, y_train_pred, squared=False))
    print("test: ", mean_squared_error(y_val, y_pred, squared=False))

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models/pickle")

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


train:  4.819813900122036
test:  5.995041447869019


FileNotFoundError: [Errno 2] No such file or directory: 'models/lin_reg.bin'

In [9]:
lr_model = Lasso()
lr_model.fit(X_train, y_train)

y_train_pred = lr_model.predict(X_train)
y_pred = lr_model.predict(X_val)

print("train: ", mean_squared_error(y_train, y_train_pred, squared=False))
print("test: ", mean_squared_error(y_val, y_pred, squared=False))

train:  9.019532867192334
test:  9.110340896856446


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [10]:
lr_model = Ridge()
lr_model.fit(X_train, y_train)

y_train_pred = lr_model.predict(X_train)
y_pred = lr_model.predict(X_val)

print("train: ", mean_squared_error(y_train, y_train_pred, squared=False))
print("test: ", mean_squared_error(y_val, y_pred, squared=False))

train:  5.155045476939375
test:  5.980948972923878


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [11]:
xgb_model = XGBRegressor()
xgb_model.fit(X_train, y_train)

y_train_pred = xgb_model.predict(X_train)
y_pred = xgb_model.predict(X_val)

print("train: ", mean_squared_error(y_train, y_train_pred, squared=False))
print("test: ", mean_squared_error(y_val, y_pred, squared=False))

train:  5.063927633534167
test:  5.365642179025045


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [12]:
with open('models/xgb_model.bin', 'wb') as f_out:
    pickle.dump((dv, xgb_model), f_out)

In [17]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [19]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}
    
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=5,
    trials=Trials()
)

[0]	validation-rmse:5.60753                                                                                                                                                                        
[1]	validation-rmse:5.56120                                                                                                                                                                        
[2]	validation-rmse:5.54220                                                                                                                                                                        
[3]	validation-rmse:5.53173                                                                                                                                                                        
[4]	validation-rmse:5.52418                                                                                                                                                                        
[5]	validation-rmse:

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:21:12] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[11]	validation-rmse:5.45669                                                                                                                                                                       
[12]	validation-rmse:5.45404                                                                                                                                                                       
[13]	validation-rmse:5.43359                                                                                                                                                                       
[14]	validation-rmse:5.42977                                                                                                                                                                       
[15]	validation-rmse:5.42257                                                                                                                                                                       
[16]	validation-rmse

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:21:28] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.38186                                                                                                                                                                        
[1]	validation-rmse:7.78073                                                                                                                                                                        
[2]	validation-rmse:7.28996                                                                                                                                                                        
[3]	validation-rmse:6.89295                                                                                                                                                                        
[4]	validation-rmse:6.57398                                                                                                                                                                        
[5]	validation-rmse:

...

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:23:28] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.68486                                                                                                                                                                        
[1]	validation-rmse:6.78087                                                                                                                                                                        
[2]	validation-rmse:6.23076                                                                                                                                                                        
[3]	validation-rmse:5.90494                                                                                                                                                                        
[4]	validation-rmse:5.71766                                                                                                                                                                        
[5]	validation-rmse:

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:24:22] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.70308                                                                                                                                                                        
[1]	validation-rmse:8.33530                                                                                                                                                                        
[2]	validation-rmse:8.00391                                                                                                                                                                        
[3]	validation-rmse:7.70690                                                                                                                                                                        
[4]	validation-rmse:7.43976                                                                                                                                                                        
[5]	validation-rmse:

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [25]:

params = {
    'learning_rate': 0.07127489236504199,
    'max_depth': 13,
    'min_child_weight': 4.483533060346516,
    'objective': 'reg:linear',
    'reg_alpha': 0.08811330031423091,
    'reg_lambda': 0.0984331190740837,
    'seed': 42
}
mlflow.xgboost.autolog()
with mlflow.start_run():

    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

2024/11/02 13:57:27 WARNING mlflow.utils.autologging_utils: MLflow xgboost autologging is known to be compatible with 1.4.2 <= xgboost <= 2.1.1, but the installed version is 2.1.2. If you encounter errors during autologging, try upgrading / downgrading xgboost to a compatible version, or try upgrading MLflow.


[0]	validation-rmse:8.70308


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:57:28] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[1]	validation-rmse:8.33530
[2]	validation-rmse:8.00391
[3]	validation-rmse:7.70690
[4]	validation-rmse:7.43976
[5]	validation-rmse:7.20100
[6]	validation-rmse:6.98852
[7]	validation-rmse:6.79770
[8]	validation-rmse:6.62908
[9]	validation-rmse:6.47849
[10]	validation-rmse:6.34689
[11]	validation-rmse:6.22930
[12]	validation-rmse:6.12515
[13]	validation-rmse:6.03322
[14]	validation-rmse:5.95198
[15]	validation-rmse:5.88096
[16]	validation-rmse:5.81792
[17]	validation-rmse:5.76305
[18]	validation-rmse:5.71438
[19]	validation-rmse:5.67159
[20]	validation-rmse:5.63407
[21]	validation-rmse:5.60051
[22]	validation-rmse:5.57121
[23]	validation-rmse:5.54627
[24]	validation-rmse:5.52288
[25]	validation-rmse:5.50242
[26]	validation-rmse:5.48433
[27]	validation-rmse:5.46841
[28]	validation-rmse:5.45336
[29]	validation-rmse:5.44008
[30]	validation-rmse:5.42929
[31]	validation-rmse:5.41800
[32]	validation-rmse:5.40912
[33]	validation-rmse:5.40099
[34]	validation-rmse:5.39310
[35]	validation-rmse:5.

2024/11/02 13:58:02 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/11/02 13:58:02 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:58:02] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2024/11/02 13:58:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calcu

In [27]:
logged_model = 'runs:/50295926e8f449aa9335b85f5008c4e0/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

loaded_model.predict(X_val)

array([19.03168  , 28.347687 ,  6.7257147, ..., 32.384693 , 13.34474  ,
       19.698849 ], dtype=float32)

In [28]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.xgboost
  run_id: 50295926e8f449aa9335b85f5008c4e0

In [30]:
xgboost_model_load = mlflow.xgboost.load_model(logged_model)
xgboost_model_load

In [32]:
xgboost_model_load.predict(valid)

array([19.03168  , 28.347687 ,  6.7257147, ..., 32.384693 , 13.34474  ,
       19.698849 ], dtype=float32)